# Analyse the quality scores of reads *after* they are mapped

Information about mapped reads is stored within the **S**equence **A**lignment **M**ap file format (SAM). Following the header, each line of a SAM file contains information about a single alignment*. Each line has eleven standardised, tab-separated, fields. See [here](https://en.wikipedia.org/wiki/SAM_(file_format) for a description of each field.

Here, we want to explore how read quality varies *along* a region of the reference genome.


*Note that since in some cases a single read can align multiple times, alignments are different than the number of reads

Written by Jason A. Hendry

In [ ]:
import os
import sys

import re

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
plt.rcParams["figure.dpi"] = 150

## Extract alignments within region of interest

We are working with *P.f.* amplicon data in this example. We will focus on the analysis on a single amplicon, *Kelch13*. In order to do this, we will extract only those alignments that fall within the *Kelch13* exon. This can be achieved using `samtools view`:

In [ ]:
data_dir = "data" # data directory
bam_fn = "BC01.sorted.bam"  # BAM file containing the alignments of interest
target_bed = "KELCH13.exons.bed"  # a BED file delineating region of interest
output_sam_fn = "BC01.K13.sam"
output_dir = "outputs"

In [ ]:
cmd = "samtools view %s -L %s -o %s" % (
    os.path.join(data_dir, bam_fn),
    os.path.join(data_dir, target_bed),
    os.path.join(output_dir, output_sam_fn))

In [ ]:
print("Command: %s" % cmd)

Above shows the samtools command we want to run. We can run this from python using an `os.system()` call:

In [ ]:
os.listdir(output_dir) # before command

In [ ]:
os.system(cmd)

In [ ]:
os.listdir(output_dir)

We have generated the desired SAM file.

## Load SAM file

In [ ]:
sam = open(os.path.join(output_dir, output_sam_fn), "r")

In [ ]:
sam.readline()

In [ ]:
# the SAM file is tab delineated, we can parse out fields with a split on tab
sam.readline().split("\t")

## Parsing the CIGAR string

Our goal is to assign each quality score to a position in the genome. If the read was mapped in one continguous region, this would be easy: we could simply align the quality scores with the start position of the read mapping. However, typically there will be many deletions and insertions in the mapping. This is especially true for nanopore data. 

Information about the mapping, including information about insertions or deletions, is stored within the CIGAR string. Briefly, the CIGAR string indicates with a character whether there has been a match "M", an insertion "I", or a deletion "D"; as well as indicating whether the read has been 'clipped' ("S" or "H"). Clipping can occur if, for example, you have limited the the SAM file to reads only present in a particular region. After the character, an integer is given which indicating the length of match, insertion, or deletion. Some examples

- 28M would indicate that there is a match of 28 bases
    - The read and the reference sequence are the same length for this region
- 10D would indicate that there is a *deletion* in the read of 10 bases
    - So the read is *shorter* than the reference genome in this region
- 2I would indicate that there is an *insertion* in the read of 2 bases
    - So the read is *longer* than the reference genome in this region


For complete details see [here](https://samtools.github.io/hts-specs/SAMv1.pdf).

In [ ]:
alignment = sam.readline().split("\t")
start = int(alignment[3])
cigar = alignment[5]
quals = alignment[10]

In [ ]:
start

In [ ]:
cigar

Now what we need to do is produce a vector of positions for each quality score, using the start position and cigar string. The basic approach is:

- If there is a match, the positions increment one base at a time
- If there is a insertion in the read, those positions in the read are *not* in the reference genome
    - Give them an invalid value (-999)
- If there is a deletion in the read, we pass
    - We are effectively skipping positions
- If there is a soft-clip, those positions in the read are *not* in the target region
    - Give them an invalid value (-998)
- If there is a hard-clip, we pass

An important detail to note here is that all the alignments within the SAM file are represented with respect to the *forward* strand. Those reads that mapped to the reverse strand are reverse-complimented before being stored in the SAM file.

In [ ]:
pos = []
i = start
tags = re.findall("\d*[MIDSH]", cigar)  # parse the cigar string into 'tags'
for tag in tags:
    n = int(tag[:-1]) # this is the integer value of the tag
    m = tag[-1] # this is the 'm'ethod: M, D, I, S, H ...
    if m == 'M':  # match
        pos.extend(np.arange(i, i + n))
    elif m == "I":  # insertion
        pos.extend(np.repeat(-999, n))
    elif m == "D":  # deletion
        pass
    elif m == "S":  # clip
        pos.extend(np.repeat(-998, n))
    elif m == "H":
        pass
    else:
        Print("Tag %s not identified." % m)
    i += n  # we have moved forward n positions
pos = np.array(pos) # convert to an array at the end

In [ ]:
print("Length of positions: %d" % len(pos))
print("Length of quality scores: %d" % len(quals))

Good. Now let's convert the quality scores to error probabilities...

In [ ]:
qs = np.array([ord(c) - 33 for c in quals])
error_probs = 10 ** (qs / -10)

In [ ]:
error_probs

Let's plot the results for this read...

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 4))

ax.plot(pos[pos > 0], error_probs[pos > 0])
ax.set_xlabel("Genomic Position")
ax.set_ylabel("Error Probability")
ax.set_title("Single-read error probabilities mapped to $K13$", loc='left')

## Generalising

Now all we need to do is expand this to handle the complete SAM file. For this we will build some functions.

In [ ]:
def positional_error_from_sam(sam_path):
    """
    Extract error probabilities and their positions
    from a SAM file
    
    params
        sam_path : str
            Path to SAM file.
            
    returns
        positions : list of ndarray, int, shape(n_reads, )
            List of numpy arrays. Each array
            encodes positions of aligned bases
            for a read. Note that -998 indicates clipped,
            -999 indicates deletion.
        error_probs : list of ndarray, float, shape(n_reads, )
            List of numpy arrays. Each array
            encodes error probabilities aligned bases
            for a read.
    
    """
    # Open SAM
    with open(sam_path, "r") as sam:

        # Prepare storage
        positions = []
        error_probs = []

        for alignment in sam:
            # Parse alignment & get relevant fields
            fields = alignment.split("\t")
            start = int(fields[3])
            cigar = fields[5]
            quals = fields[10]

            # Compute
            alignment_error_probs = calc_error_probabilities(quals)
            alignment_positions = get_positions_from_cigar(start, cigar)

            # Store
            positions.append(alignment_positions)
            error_probs.append(alignment_error_probs)
            
    return positions, error_probs

In [ ]:
def calc_error_probabilities(quals):
    """
    Calculate error probabilities from
    a string of ASCII characters `quals` that
    encode error probabilities
    
    params
        quals : str
            Error probabilities encoded in ASCII.
    
    returns
        error_probs : ndarray, float, shape(read_length, )
            Error probabilities in a
            numpy array.
    
    """
    qs = np.array([ord(c) for c in quals]) - 33
    error_probs = 10 ** (qs / -10)
    return error_probs

In [ ]:
def get_positions_from_cigar(start, cigar):
    """
    Get alignment positions for each base in a read
    given a `start` position and a `cigar` string
    
    params
        start : int
            The start position of the aligment.
        cigar : str
            The CIGAR string for the alignment.
    
    returns
        pos : ndarray, int, shape (read_length,)
            A genomic position for each base in the
            read.
    
    """
    pos = []
    i = start
    tags = re.findall("\d*[MIDSH]", cigar)  # parse the cigar string into 'tags'
    
    for tag in tags:
        n = int(tag[:-1]) # this is the integer value of the tag
        m = tag[-1] # this is the 'm'ethod: M, D, I, S, H ...
        if m == 'M':  # match
            pos.extend(np.arange(i, i + n))
        elif m == "I":  # insertion
            pos.extend(np.repeat(-999, n))
        elif m == "D":  # deletion
            pass
        elif m == "S":  # clip
            pos.extend(np.repeat(-998, n))
        elif m == "H":
            pass
        else:
            Print("Tag %s not identified." % m)
        i += n  # we have moved forward n positions
    
    return np.array(pos)

In [ ]:
sam_path = os.path.join(output_dir, output_sam_fn)
positions, error_probs = positional_error_from_sam(sam_path)

In [ ]:
# The number of reads...
len(positions)

In [ ]:
len(error_probs)

## Munge and visualise results

In [ ]:
positions = np.concatenate(positions)
error_probs = np.concatenate(error_probs)

In [ ]:
keep = positions > 0
positions = positions[keep]
error_probs = error_probs[keep]

In [ ]:
len(positions)
# 776 thousand bases have a quality score

In [ ]:
df = (pd.DataFrame({"position": positions, "error_prob": error_probs})
      .groupby("position")
      .mean()
      .reset_index()
      .sort_values("position")
     )

In [ ]:
df.head() # mean error probabilities at each position

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 4))

ax.plot(df["position"], df["error_prob"])
ax.set_xlabel("Genomic Position")
ax.set_ylabel("Error Probability")
ax.set_title("Mean Error Probability Across $Kelch13$", loc="left")

- Looks like more error at one end.. less coverage there perhaps?

In [ ]:
df["coverage"] = (pd.DataFrame({"position": positions, "error_prob": error_probs})
                  .groupby("position")
                  .size()
                  .values
                 )

In [ ]:
df.head()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 4))

ax.plot(df["position"], df["error_prob"])
ax.set_xlabel("Genomic Position")
ax.set_ylabel("Error Probability")
ax.set_title("Mean Error Probability Across $Kelch13$", loc="left")

axm = ax.twinx()
axm.fill_between(x=df["position"],
                 y1=np.repeat(0, df.shape[0]),
                 y2=df["coverage"],
                 color='darkgrey', alpha=0.5)
axm.set_ylabel("Coverage")

- Result seems to be explain by a decline in coverage
- Would be interesting to indicate primer positions and gene body